In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/Fourier_Data

# 1. 1D problem: the (time-independent) Burgers equation

## 1.1. MWT_1D

In [ ]:
from utils import train, test, LpLoss, get_filter, UnitGaussianNormalizer

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data as data_utils
from typing import List, Tuple

import sys
import os

import numpy as np
import math
from scipy.io import loadmat, savemat
import matplotlib.pyplot as plt
from scipy.special import eval_legendre
from sympy import Poly, legendre, Symbol
import h5py


import operator
from functools import reduce
from functools import partial
from timeit import default_timer

In [ ]:
# torch.manual_seed(0)
# np.random.seed(0)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Coupled Data

ntrain = 800
ntest = 200

sub = 2**0 #subsampling rate
h = 2**10 // sub #total grid size divided by the subsampling rate
s = h


batch_size = 20

rw_u = loadmat('/content/drive/MyDrive/gray_scott_results/Coupled_PDE_data/kernel1Drho_t0_1.mat')
x_data = rw_u['rho_t0'].astype(np.float32)
y_data = rw_u['rho_t02'].astype(np.float32)
print(x_data.shape)

x_train_u = x_data[:ntrain,::sub]
y_train_u = y_data[:ntrain,::sub]
x_test_u = x_data[-ntest:,::sub]
y_test_u = y_data[-ntest:,::sub]

x_train_u = torch.from_numpy(x_train_u)
x_test_u = torch.from_numpy(x_test_u)
y_train_u = torch.from_numpy(y_train_u)
y_test_u = torch.from_numpy(y_test_u)

x_train_u = x_train_u.unsqueeze(-1)
x_test_u = x_test_u.unsqueeze(-1)


rw_v = loadmat('/content/drive/MyDrive/gray_scott_results/Coupled_PDE_data/kernel1Dphi_t0_1.mat')
x_data = rw_v['phi_t0'].astype(np.float32)
y_data = rw_v['phi_t02'].astype(np.float32)

x_train_v = x_data[:ntrain,::sub]
y_train_v = y_data[:ntrain,::sub]
x_test_v = x_data[-ntest:,::sub]
y_test_v = y_data[-ntest:,::sub]

x_train_v = torch.from_numpy(x_train_v)
x_test_v = torch.from_numpy(x_test_v)
y_train_v = torch.from_numpy(y_train_v)
y_test_v = torch.from_numpy(y_test_v)
print(y_test_u.shape)

x_train_v = x_train_v.unsqueeze(-1)
x_test_v = x_test_v.unsqueeze(-1)

x_train = torch.cat([x_train_u.reshape(ntrain,s,1), x_train_v.reshape(ntrain,s,1)], dim=2)
x_test = torch.cat([x_test_u.reshape(ntest,s,1), x_test_v.reshape(ntest,s,1)], dim=2)

y_train = torch.cat([y_train_u.reshape(ntrain,s,1), y_train_v.reshape(ntrain,s,1)], dim=2)
y_test = torch.cat([y_test_u.reshape(ntest,s,1), y_test_v.reshape(ntest,s,1)], dim=2)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test[:,:,0].shape)

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)

(1000, 1024)
torch.Size([200, 1024])
torch.Size([800, 1024, 2])
torch.Size([200, 1024, 2])
torch.Size([800, 1024, 2])
torch.Size([200, 1024])


In [ ]:
# with open('data_no_couple/x_train.npy', 'wb') as f:
#     np.save(f, np.array(x_train))

In [ ]:
# with open('data_no_couple/x_test.npy', 'wb') as f:
#     np.save(f, np.array(x_test))
# with open('data_no_couple/y_train.npy', 'wb') as f:
#     np.save(f, np.array(y_train))
# with open('data_no_couple/y_test.npy', 'wb') as f:
#     np.save(f, np.array(y_test))

In [ ]:
def get_initializer(name):
    
    if name == 'xavier_normal':
        init_ = partial(nn.init.xavier_normal_)
    elif name == 'kaiming_uniform':
        init_ = partial(nn.init.kaiming_uniform_)
    elif name == 'kaiming_normal':
        init_ = partial(nn.init.kaiming_normal_)
    return init_

In [ ]:
x_train.shape

torch.Size([800, 1024, 2])

In [ ]:
class sparseKernel(nn.Module):
    def __init__(self,
                 k, alpha, c=1,
                 nl = 1,
                 initializer = None,
                 **kwargs):
        super(sparseKernel,self).__init__()
       
        self.k = k
        self.Li = nn.Linear(c*k, 128)
        self.conv = self.convBlock(c*k, 128)
#         self.Lo = nn.Linear(alpha*k, c*k)
        self.Lo = nn.Linear(128, c*k)
       
    def forward(self, x):
        B, N, c, ich = x.shape # (B, N, c, k)
        x = x.view(B, N, -1)
#         x = F.relu(self.Li(x))
        x = x.permute(0, 2, 1)
        x = self.conv(x)
        x = x.permute(0, 2, 1)
        x = self.Lo(x)
        x = x.view(B, N, c, ich)
       
        return x
       
       
    def convBlock(self, ich, och):
        net = nn.Sequential(
            nn.Conv1d(ich, och, 3, 1, 1),
            nn.ReLU(inplace=True),
#             nn.Conv1d(och, och, 3, 1, 1),
#             nn.ReLU(inplace=True),
        )
        return net

def compl_mul1d(x, weights):
    # (batch, in_channel, x ), (in_channel, out_channel, x) -> (batch, out_channel, x)
    return torch.einsum("bix,iox->box", x, weights)

class sparseKernelFT(nn.Module):
    def __init__(self,
                 k, alpha, c=1,
                 nl = 1,
                 initializer = None,
                 **kwargs):
        super(sparseKernelFT, self).__init__()       
       
        self.modes1 = alpha
        self.scale = (1 / (c*k*c*k))
        self.weights1 = nn.Parameter(self.scale * torch.rand(c*k, c*k, self.modes1, dtype=torch.cfloat))
        self.weights1.requires_grad = True
        self.k = k
       
    def forward(self, x):
        B, N, c, k = x.shape # (B, N, c, k)
       
        x = x.view(B, N, -1)
        x = x.permute(0, 2, 1)
        x_fft = torch.fft.rfft(x)
        # Multiply relevant Fourier modes
        l = min(self.modes1, N//2+1)
        out_ft = torch.zeros(B, c*k, N//2 + 1,  device=x.device, dtype=torch.cfloat)
       
        out_ft[:, :, :l] = compl_mul1d(x_fft[:, :, :l], self.weights1[:, :, :l])
       

        #Return to physical space
        x = torch.fft.irfft(out_ft, n=N)
        x = x.permute(0, 2, 1).view(B, N, c, k)
        return x
       
   
class MWT_CZ(nn.Module):
    def __init__(self,
                 k = 3, alpha = 5,
                 L = 0, c = 1,
                 base = 'legendre',
                 initializer = None,
                 **kwargs):
        super(MWT_CZ, self).__init__()
       
        self.k = k
        self.L = L
        H0, H1, G0, G1 = get_filter(base, k)
       
        self.A = sparseKernelFT(k, alpha, c)
        self.B = sparseKernelFT(k, alpha, c)
        self.C = sparseKernelFT(k, alpha, c)
       
        self.T0 = nn.Linear(k, k)

        self.register_buffer('ec_s', torch.Tensor(
            np.concatenate((H0.T, H1.T), axis=0)))
        self.register_buffer('ec_d', torch.Tensor(
            np.concatenate((G0.T, G1.T), axis=0)))
       
        self.register_buffer('rc_e', torch.Tensor(
            np.concatenate((H0, G0), axis=0)))
        self.register_buffer('rc_o', torch.Tensor(
            np.concatenate((H1, G1), axis=0)))
       
       
    def forward(self, x, u_d = None, u_s = None):
       
        B, N, c, ich = x.shape # (B, N, k)
        ns = math.floor(np.log2(N))

        Ud = torch.jit.annotate(List[Tensor], [])
        Us = torch.jit.annotate(List[Tensor], [])
#         decompose
        for i in range(ns-self.L):
            d, x = self.wavelet_transform(x)
            Ud += [self.A(d) + self.B(x)]
            Us += [self.C(d)]
        x = self.T0(x) # coarsest scale transform
        #print('x = self.T0(x)',x.shape)

        if u_d == None:
            for i in range(ns-1-self.L,-1,-1):
                x = x + Us[i]
                x = torch.cat((x, Ud[i]), -1) # 还没加Ud
                x = self.evenOdd(x)
                #print('x_none', x.shape)
            return x, Ud, Us

        else:
#        reconstruct 

            for i in range(ns-1-self.L,-1,-1):
                x = x + Us[i] + u_s[i]
                x = torch.cat((x, Ud[i]), -1) # 还没加Ud
                x = self.evenOdd(x)
                #print('x_us', x.shape)
            return x, Ud, Us

   
    def wavelet_transform(self, x):
        xa = torch.cat([x[:, ::2, :, :],
                        x[:, 1::2, :, :],
                       ], -1)
        d = torch.matmul(xa, self.ec_d)
        s = torch.matmul(xa, self.ec_s)
        return d, s
       
       
    def evenOdd(self, x):
       
        B, N, c, ich = x.shape # (B, N, c, k)
        # print(ich)
        # print(self.k)
        assert ich == 2*self.k
        x_e = torch.matmul(x, self.rc_e)
        x_o = torch.matmul(x, self.rc_o)
       
        x = torch.zeros(B, N*2, c, self.k,
            device = x.device)
        x[..., ::2, :, :] = x_e
        x[..., 1::2, :, :] = x_o
        return x

    def evenOdd_2(self, x):
       
        B, N, c, ich = x.shape # (B, N, c, k)
        # print(ich)
        # print(self.k)
        assert ich == 2*self.k
        x_e = torch.matmul(x, self.rc_e)
        x_o = torch.matmul(x, self.rc_o)
       
        x = torch.zeros(B, N*2, c, self.k,
            device = x.device)
        x[..., ::2, :, :] = x_e
        x[..., 1::2, :, :] = x_o
        return x
   
   
class MWT(nn.Module):
    def __init__(self,
                 ich = 1, k = 3, alpha = 2, c = 1,
                 nCZ = 3,
                 L = 0,
                 base = 'legendre',
                 initializer = None,
                 **kwargs):
        super(MWT,self).__init__()
       
        self.k = k
        self.c = c
        self.L = L
        self.nCZ = nCZ
        self.Lk = nn.Linear(ich, c*k)
       
        self.MWT_CZ = nn.ModuleList(
            [MWT_CZ(k, alpha, L, c, base,
            initializer) for _ in range(nCZ)]
        )
        self.Lc0 = nn.Linear(c*k, 128)
        self.Lc1 = nn.Linear(128, 1)
       
        if initializer is not None:
            self.reset_parameters(initializer)
       
    def forward(self, x, u_d = None, u_s = None):
       
        B, N, ich = x.shape # (B, N, d)
        ns = math.floor(np.log2(N))
        x = self.Lk(x)
        x = x.view(B, N, self.c, self.k)

        Ud = torch.jit.annotate(List[Tensor], [])
        Us = torch.jit.annotate(List[Tensor], [])
        if u_d == None:
            for i in range(self.nCZ):
                x, ud, us = self.MWT_CZ[i](x)
                x = torch.tanh(x)
                Ud += [ud]
                Us += [us]
            x = x.view(B, N, -1) # collapse c and k
            x = self.Lc0(x)
            x = F.relu(x)
            x = self.Lc1(x)
            #print('MWT', x.shape)
            return x, Ud, Us
          
        else:
            for i in range(self.nCZ):
                x, ud, us = self.MWT_CZ[i](x, u_d[i], u_s[i])
                x = torch.tanh(x)
                Ud += [ud]
                Us += [us]
            x = x.view(B, N, -1) # collapse c and k
            x = self.Lc0(x)
            x = F.relu(x)
            x = self.Lc1(x)
            return x, Ud, Us
   
    def reset_parameters(self, initializer):
        initializer(self.Lc0.weight)
        initializer(self.Lc1.weight)

In [ ]:
# New model
ich = 2
initializer = get_initializer('xavier_normal') # xavier_normal, kaiming_normal, kaiming_uniform

# torch.manual_seed(0)
# np.random.seed(0)
model1 = MWT(ich,
            alpha = 10,
            c = 4*4,
            k = 4,
            base = 'legendre',
            nCZ = 2,
            initializer = initializer,
            ).to(device)

model2 = MWT(ich,
            alpha = 10,
            c = 4*4,
            k = 4,
            base = 'legendre',
            nCZ = 2,
            initializer = initializer,
            ).to(device)
learning_rate = 0.001

epochs = 500
step_size = 100
gamma = 0.5

In [ ]:
################################################################
# training and evaluation
################################################################
import random

optimizer1 = torch.optim.Adam(model1.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler1 = torch.optim.lr_scheduler.StepLR(optimizer1, step_size=step_size, gamma=gamma)

optimizer2 = torch.optim.Adam(model2.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler2 = torch.optim.lr_scheduler.StepLR(optimizer2, step_size=step_size, gamma=gamma)

In [ ]:
# Train and test
error_u = []
error_v = []
mae_error_u = []
mae_error_v = []

myloss = LpLoss(size_average=False)
mse = nn.MSELoss()
mae = nn.L1Loss()
len_loader = ntrain/batch_size
print(len_loader)


for ep in range(epochs):
    model1.train()
    model2.train()
    t1 = default_timer()

    train_l2_u = 0
    train_l2_v = 0

    flag_1 = 0
    flag_2 = 0
    '''
    for x, y in train_loader:
      x, y = x.cuda(), y.cuda()

      random_num = random.random()
      # NO1->NO2
      if random_num < 0.5:

        # if flag_1 > len_loader/2:
        #   continue

        # else:
          u_train_mid = torch.cat([x,  torch.zeros([batch_size,s,1]).cuda()], dim=2)
          #print(u_train_mid.shape)
          with torch.no_grad(): # No grad upgrade

            #u_out_mid = model1(u_train_mid.detach())
            u_out_mid, Ud, Us = model1(x)

          #v_train_end = torch.cat([x,  u_out_mid], dim=2)
            

          optimizer2.zero_grad()

          #v_train_end = torch.cat([x], dim=2)

          v_out_end, Ud, Us = model2(x, Ud, Us)

          l2_v = myloss(v_out_end.view(batch_size, -1), y[:,:,1].view(batch_size, -1))
          l2_v.backward() # use the l2 relative loss

          optimizer2.step()
          train_l2_v += l2_v.item()

          flag_1 += 1
      
      # NO2->NO1
      else: 
        # if flag_2 > len_loader/2:
        #   continue
        
        # else:

          v_train_mid =  torch.cat([x,  torch.zeros([batch_size,s,1]).cuda()], dim=2)
          with torch.no_grad():
            v_out_mid, Ud, Us = model2(x)
            # print(len(Ud))
            # print(Ud[0])
          # print(x.shape)
          # print(u_out_mid.shape)
          #v_train_end = torch.cat([x,  u_out_mid], dim=2)
          
          
          optimizer1.zero_grad()

          #u_train_end = torch.cat([x,  v_out_mid], dim=2)
          u_out_end, Ud, Us = model1(x, Ud, Us)
          l2_u = myloss(u_out_end.view(batch_size, -1), y[:,:,0].view(batch_size, -1))
          l2_u.backward() # use the l2 relative loss

          optimizer1.step()
          train_l2_u += l2_u.item()

          flag_2 += 1
      '''

      for x, y in train_loader:
        x, y = x.cuda(), y.cuda()

        random_num = random.random()
        # NO1->NO2
        if random_num < 0.5:

          u_train_mid = torch.cat([x,  torch.zeros([batch_size,s,1]).cuda()], dim=2)
          with torch.no_grad(): # No grad upgrade
              u_out_mid, Ud, Us = model1(x)
          optimizer2.zero_grad()
          v_out_end, Ud, Us = model2(x, Ud, Us)
          l2_v = myloss(v_out_end.view(batch_size, -1), y[:,:,1].view(batch_size, -1))
          l2_v.backward() # use the l2 relative loss
          optimizer2.step()
          train_l2_v += l2_v.item()

          v_train_mid =  torch.cat([x,  torch.zeros([batch_size,s,1]).cuda()], dim=2)
          with torch.no_grad():
              v_out_mid, Ud, Us = model2(x)
          optimizer1.zero_grad()
          u_out_end, Ud, Us = model1(x, Ud, Us)
          l2_u = myloss(u_out_end.view(batch_size, -1), y[:,:,0].view(batch_size, -1))
          l2_u.backward() # use the l2 relative loss
          optimizer1.step()
          train_l2_u += l2_u.item()

        # NO2->NO1
        else: 

          v_train_mid =  torch.cat([x,  torch.zeros([batch_size,s,1]).cuda()], dim=2)
          with torch.no_grad():
              v_out_mid, Ud, Us = model2(x)
          optimizer1.zero_grad()
          u_out_end, Ud, Us = model1(x, Ud, Us)
          l2_u = myloss(u_out_end.view(batch_size, -1), y[:,:,0].view(batch_size, -1))
          l2_u.backward() # use the l2 relative loss
          optimizer1.step()
          train_l2_u += l2_u.item()

          u_train_mid = torch.cat([x,  torch.zeros([batch_size,s,1]).cuda()], dim=2)
          with torch.no_grad(): # No grad upgrade
              u_out_mid, Ud, Us = model1(x)
          optimizer2.zero_grad()
          v_out_end, Ud, Us = model2(x, Ud, Us)
          l2_v = myloss(v_out_end.view(batch_size, -1), y[:,:,1].view(batch_size, -1))
          l2_v.backward() # use the l2 relative loss
          optimizer2.step()
          train_l2_v += l2_v.item()
            



# # Test:
    model1.eval()
    model2.eval()

    scheduler1.step()
    scheduler2.step()

    test_l2_u = 0.0
    i = 0
    for x, y in test_loader:
        x, y = x.cuda(), y.cuda()
        #v_test_mid = torch.cat([x,  torch.zeros([batch_size,s,1]).cuda()], dim=2)
        v_test_out_mid, Ud, Us = model2(x)
        #u_test_end = torch.cat([x,  v_test_out_mid], dim=2)
        u_test_out_end, Ud, Us = model1(x,Ud, Us)
        u_test_out_end = u_test_out_end.view(batch_size, -1)
        y = y[:,:,0].view(batch_size, -1)
        if i == 0:
            u_pred =  u_test_out_end
            u_label = y
            i += 1
        else:
            u_pred = torch.cat((u_pred, u_test_out_end),0)
            u_label = torch.cat((u_label, y), 0)
        
        test_l2_u += myloss(u_test_out_end, y).item()

    train_l2_u /= ntrain
    test_l2_u /= ntest
    error_u.append(test_l2_u)
#     mse_u = mse(u_pred, u_label)
#     rmse_u = torch.sqrt(mse_u)
    mae_u = mae(u_pred, u_label)

    test_l2_v = 0.0
    i = 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.cuda(), y.cuda()

            #u_test_mid = torch.cat([x,  torch.zeros([batch_size,s,1]).cuda()], dim=2)
            u_test_out_mid, Ud, Us = model1(x)
            #v_test_end = torch.cat([x,  u_test_out_mid], dim=2)
            v_test_out_end, Ud, Us = model2(x, Ud, Us)
            
            v_test_out_end = v_test_out_end.view(batch_size, -1)
            y = y[:,:,1].view(batch_size, -1)
            if i == 0:
                v_pred =  v_test_out_end
                v_label = y
                i += 1
            else:
                v_pred = torch.cat((v_pred, v_test_out_end),0)
                v_label = torch.cat((v_label, y), 0)
            test_l2_v += myloss(v_test_out_end, y).item()

    train_l2_v /= ntrain
    test_l2_v /= ntest
    error_v.append(test_l2_v)
#     mse_v = mse(v_pred, v_label)
#     rmse_v = torch.sqrt(mse_v)
    mae_v = mae(v_pred, v_label)
    mae_error_u.append(mae_u.item())
    mae_error_v.append(mae_v.item())


    print(ep,train_l2_u, train_l2_v,test_l2_u, test_l2_v,mae_u.item(),mae_v.item())

np.save('/content/drive/MyDrive/gray_scott_results/Gray_scott_u_l0l0_ugrf_vgrf_SOTA_error',error_u)
np.save('/content/drive/MyDrive/gray_scott_results/Gray_scott_v_l0l0_ugrf_vgrf_SOTA_error',error_v)
np.save('/content/drive/MyDrive/gray_scott_results/Gray_scott_u_l0l0_ugrf_vgrf_SOTA_mae',mae_error_u)
np.save('/content/drive/MyDrive/gray_scott_results/Gray_scott_v_l0l0_ugrf_vgrf_SOTA_mae',mae_error_v)

40.0
0 0.24808538049459458 0.23404061317443847 0.1660042929649353 0.16820290684700012 19 21
1 0.08292121708393096 0.06217746794223786 0.12341262578964234 0.10481873631477356 17 23
2 0.05275621771812439 0.04041439786553383 0.07439816832542419 0.07072568535804749 18 22
3 0.0510540959239006 0.02572245627641678 0.08290363013744355 0.0663626265525818 15 25
4 0.03624243035912514 0.03524769738316536 0.07773786783218384 0.05264899879693985 23 17
5 0.04359373554587364 0.02615192160010338 0.05301964998245239 0.05877300262451172 19 21
6 0.03323824241757393 0.02187147840857506 0.050553068816661835 0.04257237076759338 19 21
7 0.03145403146743774 0.022631964907050134 0.06407972276210785 0.0485115385055542 20 20
8 0.03477469876408577 0.020789886116981505 0.057112912833690646 0.039789007008075715 19 21
9 0.022489354088902472 0.028375535383820535 0.05333832859992981 0.04473869442939758 25 15
10 0.029798105359077454 0.01704562149941921 0.042385170459747316 0.05174964100122452 16 24
11 0.0213088882714509

91 0.01256319709122181 0.01952333275228739 0.03503961086273193 0.027362726628780365 26 14
92 0.018468688651919366 0.016090968325734138 0.03821983695030212 0.029526235163211824 22 18
93 0.024968395233154295 0.012623917795717716 0.04479918569326401 0.025931684672832488 18 22
94 0.022602590397000313 0.01335619680583477 0.051482360363006595 0.02452677384018898 20 20
95 0.016623404920101167 0.02160247039049864 0.04670020937919617 0.028991113752126693 26 14
96 0.020236612781882286 0.015665228515863418 0.035440749526023864 0.027167800217866897 21 19
97 0.024935629777610303 0.010983601249754428 0.04325010240077973 0.032337654232978824 14 26
98 0.021276872083544732 0.01327554576098919 0.031454410254955295 0.03787076413631439 16 24
99 0.01661281496286392 0.01919791392982006 0.03278954386711121 0.03312756329774857 20 20
100 0.01309286780655384 0.010810564681887626 0.02526413157582283 0.01935099646449089 20 20
101 0.014282708764076233 0.007315990626811981 0.024547103345394134 0.01939509391784668 1

180 0.010512444116175175 0.006044730581343174 0.022424929589033128 0.015933233499526977 18 22
181 0.00813626665621996 0.007346155606210231 0.021067019253969192 0.013826128244400025 22 18
182 0.010138191357254982 0.006482642404735089 0.02036666288971901 0.013274574875831604 19 21
183 0.009254852458834648 0.006449296213686466 0.016440258026123047 0.014968543201684953 21 19
184 0.007750006373971701 0.007179536148905754 0.01838253900408745 0.027664468586444856 21 19
185 0.010990630593150854 0.007112345360219478 0.020657148212194443 0.015246567279100419 13 27
186 0.010518436469137669 0.00872484989464283 0.021452539563179017 0.01469816341996193 21 19
187 0.010454492438584565 0.00749140165746212 0.018525629937648773 0.018773097544908524 18 22
188 0.010610563978552819 0.0078027226589620115 0.02467328906059265 0.014110978841781616 20 20
189 0.01072739988565445 0.006385000329464674 0.016561760902404785 0.023550601452589037 17 23
190 0.009969261288642884 0.008065066374838352 0.01829883798956871 0

268 0.005723794531077147 0.0036576007679104804 0.015439308658242225 0.010669715330004692 17 23
269 0.007674136348068714 0.003624564055353403 0.011922425255179406 0.014053861275315285 14 26
270 0.005305014569312334 0.005444891154766083 0.012424163445830345 0.009551924020051956 21 19
271 0.0046318391896784305 0.004416774399578572 0.012218613401055336 0.01009672187268734 22 18
272 0.004573736116290092 0.004820569250732661 0.013303346708416939 0.010233909413218499 21 19
273 0.005214666426181794 0.005694612227380276 0.013442094549536704 0.012899730578064919 21 19
274 0.004777145870029927 0.006101033873856068 0.01259964108467102 0.0125922292470932 22 18
275 0.004644629657268524 0.005333538986742496 0.013287320882081986 0.010255606397986412 23 17
276 0.006805348750203848 0.004303066972643137 0.01275794729590416 0.011143491864204406 18 22
277 0.003908324670046568 0.005965116452425718 0.013352604284882545 0.009655711278319358 26 14
278 0.006486647482961416 0.003808249440044165 0.012162008434534

355 0.004943603752180934 0.002472388157621026 0.010290777012705803 0.008609583228826522 15 25
356 0.002796760108321905 0.004558908650651574 0.01099583700299263 0.00818860024213791 26 14
357 0.00416551947593689 0.0032435795199126007 0.012033376842737198 0.007696583792567253 20 20
358 0.003034762805327773 0.0040735981054604055 0.011256203949451447 0.008237756565213203 25 15
359 0.0031557362154126167 0.003890512641519308 0.009770919755101203 0.007733739241957665 24 16
360 0.0028630274999886752 0.004355333978310227 0.010067617893218994 0.00850762464106083 25 15
361 0.003828423162922263 0.0034197603072971105 0.009337407499551774 0.007697374895215035 21 19
362 0.0043930233642458915 0.0024271564185619353 0.009892198964953422 0.007813887968659401 16 24
363 0.003609725907444954 0.0029933554958552124 0.010632247254252434 0.007595236226916313 20 20
364 0.003853918667882681 0.002949226489290595 0.009461692720651626 0.007902869433164596 19 21
365 0.004390624202787876 0.0023080836888402703 0.0093688

442 0.002574257776141167 0.002940968982875347 0.008402015268802642 0.006980651989579201 22 18
443 0.0034934216178953647 0.001994940135627985 0.008257781341671943 0.007173057198524475 15 25
444 0.0029894096218049527 0.0024457524344325067 0.008449212312698365 0.007004214376211166 18 22
445 0.002813160195946693 0.0027103321719914673 0.008365206345915794 0.006970553547143936 20 20
446 0.002376237753778696 0.002920274576172233 0.00821463979780674 0.0069438529014587405 22 18
447 0.0024541858863085506 0.0030003426410257815 0.008624996095895766 0.007013280466198921 22 18
448 0.002234261818230152 0.0032428494188934563 0.008454226180911064 0.007015150636434555 24 16
449 0.003121725069358945 0.0024048429634422066 0.00865705244243145 0.006985228210687637 18 22
450 0.002786744125187397 0.0026438786555081605 0.009201166406273841 0.007041897475719452 20 20
451 0.003217737441882491 0.002427955847233534 0.008654546663165093 0.006975045129656791 18 22
452 0.0025526187382638454 0.003061288492754102 0.008